# 1.导库，获取数据

In [34]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [35]:
data = pd.read_csv('./data/rankingcard.csv',index_col = 0)

In [36]:
data.shape

(150000, 11)

In [37]:
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


# 2.探索数据与数据预处理

## 2.1去除重复值

In [ ]:
#去除重复值
data.drop_duplicates(inplace=True)
data.info()

In [40]:
#删除之后一定是要恢复索引
data.reset_index(drop=True,inplace=True)
#data.index = range(data.shape[0])
data.info()

In [42]:
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


## 2.2 填补缺失值

In [50]:
#均值填补
#data.isna().sum()/data.shape[0]
data.isna().mean()

SeriousDlqin2yrs                        0.000000
RevolvingUtilizationOfUnsecuredLines    0.000000
age                                     0.000000
NumberOfTime30-59DaysPastDueNotWorse    0.000000
DebtRatio                               0.000000
MonthlyIncome                           0.195601
NumberOfOpenCreditLinesAndLoans         0.000000
NumberOfTimes90DaysLate                 0.000000
NumberRealEstateLoansOrLines            0.000000
NumberOfTime60-89DaysPastDueNotWorse    0.000000
NumberOfDependents                      0.025624
dtype: float64

In [52]:
data['NumberOfDependents'].fillna(int(data['NumberOfDependents'].mean()),inplace=True)

In [53]:
data.isna().mean()

SeriousDlqin2yrs                        0.000000
RevolvingUtilizationOfUnsecuredLines    0.000000
age                                     0.000000
NumberOfTime30-59DaysPastDueNotWorse    0.000000
DebtRatio                               0.000000
MonthlyIncome                           0.195601
NumberOfOpenCreditLinesAndLoans         0.000000
NumberOfTimes90DaysLate                 0.000000
NumberRealEstateLoansOrLines            0.000000
NumberOfTime60-89DaysPastDueNotWorse    0.000000
NumberOfDependents                      0.000000
dtype: float64

In [65]:
#随机森林
#填补函数
def fill_missing_rf(x,y,to_fill):
    """
    使用随机森林填补一个特征的缺失值的函数
    :param x: 要填补的特征矩阵
    :param y: 完整的，没有缺失值的标签（原有标签）
    :param to_fill: 要填补的特征
    :return: 缺失填补后的特征
    """
    #构建新特征矩阵和标签
    df = x.copy()
    fill = df.loc[:,to_fill]
    df = pd.concat([df.loc[:,df.columns!=to_fill],pd.DataFrame(y)],axis=1)

    #找出训练集和测试集
    y_train = fill[fill.notnull()]
    y_test  = fill[fill.isnull()]
    x_train = df.iloc[y_train.index,:]
    x_test = df.iloc[y_test.index,:]

    #使用随机森林填补缺失值
    from sklearn.ensemble import RandomForestRegressor as rfr
    rfr = rfr(n_estimators=100)
    rfr.fit(x_train,y_train)
    yprdict = rfr.predict(x_test)

    return yprdict

In [105]:
x = data.iloc[:,1:]

In [106]:
y= data['SeriousDlqin2yrs']

In [107]:
ypredict = fill_missing_rf(x,y,'MonthlyIncome')

In [124]:
type(ypredict)

numpy.ndarray

In [71]:
to_fill = 'MonthlyIncome'
df = x.copy()
fill = df.loc[:,to_fill]
df = pd.concat([df.loc[:,df.columns!=to_fill],pd.DataFrame(y)],axis=1)

In [73]:
y_train = fill[fill.notnull()]
y_test  = fill[fill.isnull()]
x_train = df.iloc[y_train.index,:]
x_test = df.iloc[y_test.index,:]

In [109]:
y_train

0          9120.0
1          2600.0
2          3042.0
3          3300.0
4         63588.0
           ...   
149385     3400.0
149386     2100.0
149387     5584.0
149389     5716.0
149390     8158.0
Name: MonthlyIncome, Length: 120170, dtype: float64

In [125]:
data = pd.concat([y_train,pd.DataFrame(ypredict)],axis=0).reset_index(drop=True)

In [133]:
df.loc[:,'MonthlyIncome'] = data[0]


In [134]:

df.isna().mean()

RevolvingUtilizationOfUnsecuredLines    0.0
age                                     0.0
NumberOfTime30-59DaysPastDueNotWorse    0.0
DebtRatio                               0.0
MonthlyIncome                           0.0
NumberOfOpenCreditLinesAndLoans         0.0
NumberOfTimes90DaysLate                 0.0
NumberRealEstateLoansOrLines            0.0
NumberOfTime60-89DaysPastDueNotWorse    0.0
NumberOfDependents                      0.0
dtype: float64

In [112]:
y

0         1
1         0
2         0
3         0
4         0
         ..
149386    0
149387    0
149388    0
149389    0
149390    0
Name: SeriousDlqin2yrs, Length: 149391, dtype: int64

In [123]:
pd.concat([y_train,pd.DataFrame(ypredict)],axis=0).reset_index(drop=True)

TypeError: Index(...) must be called with a collection of some kind, 'MonthlyIncome' was passed

In [115]:
df.loc[:,'MonthlyIncome'] = pd.concat([y_train,pd.DataFrame(ypredict)],axis=0).reset_index(drop=True)

In [121]:
df.loc[:,'MonthlyIncome']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
149386   NaN
149387   NaN
149388   NaN
149389   NaN
149390   NaN
Name: MonthlyIncome, Length: 149391, dtype: float64